In [10]:
import os
import json
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import timm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
import matplotlib.pyplot as plt
# import seaborn as sns

# 设置随机种子，确保结果可复现
SEED = 42
torch.manual_seed(SEED)
np.random.seed(SEED)

# 1. 数据预处理 - 缺失值填充
def fill_missing_values(df, is_train=True, fill_params=None):
    """填充缺失值，训练集计算填充参数，验证/测试集使用训练集参数"""
    df_filled = df.copy()
    
    # 数值型列填充（使用中位数抗异常值）
    numeric_cols = ['townsend', 'bmi', 'baselineage']
    if is_train:
        fill_params = {}
        for col in numeric_cols:
            if col in df_filled.columns:
                median_val = df_filled[col].median()
                fill_params[f'{col}_median'] = median_val
                df_filled[col].fillna(median_val, inplace=True)
    else:
        for col in numeric_cols:
            if col in df_filled.columns and f'{col}_median' in fill_params:
                df_filled[col].fillna(fill_params[f'{col}_median'], inplace=True)
    
    # 类别型列填充（使用众数或'unknown'）
    categorical_cols = ['gender', 'smokingc', 'drinkc', 'ethnic', 'edu', 
                       'hbp', 'dmstatus', 'hyperlipidemia', 
                       'baseline_depression', 'incident_depression']
    if is_train:
        for col in categorical_cols:
            if col not in df_filled.columns:
                continue
            missing_ratio = (df_filled[col].isnull().sum() / len(df_filled)) * 100
            if missing_ratio < 50:
                # 使用众数填充
                mode_val = df_filled[col].mode()[0]
                fill_params[f'{col}_mode'] = mode_val
                df_filled[col].fillna(mode_val, inplace=True)
            else:
                # 缺失比例过高，使用'unknown'填充
                fill_params[f'{col}_fill'] = 'unknown'
                df_filled[col].fillna('unknown', inplace=True)
    else:
        for col in categorical_cols:
            if col not in df_filled.columns:
                continue
            if f'{col}_mode' in fill_params:
                df_filled[col].fillna(fill_params[f'{col}_mode'], inplace=True)
            elif f'{col}_fill' in fill_params:
                df_filled[col].fillna(fill_params[f'{col}_fill'], inplace=True)
    
    if is_train:
        return df_filled, fill_params
    else:
        return df_filled

# 2. 定义三分类标签函数
def get_depression_class(row):
    """
    三分类判断：
    - 0: 无抑郁
    - 1: 基线抑郁（baseline_depression为"是"或new_totdepress为1）
    - 2: 新发抑郁（基线无抑郁，但incident_depression为"是"）
    """
    # 基线抑郁（类别1）
    if row['baseline_depression'] == '是' or row['new_totdepress'] == 1:
        return 1
    # 新发抑郁（类别2）
    elif row['baseline_depression'] == '否' and row['new_totdepress'] == 0 and row['incident_depression'] == '是':
        return 2
    # 无抑郁（类别0）
    elif row['baseline_depression'] == '否' and row['new_totdepress'] == 0 and row['incident_depression'] == '否':
        return 0
    # 异常情况
    else:
        print(f"警告：数据异常 - 行索引{row.name}，基线抑郁:{row['baseline_depression']}, "
              f"new_totdepress:{row['new_totdepress']}, 新发抑郁:{row['incident_depression']}，暂归为无抑郁")
        return 0

# 3. 定义双模态数据集
class DualModalDataset(Dataset):
    def __init__(self, image_dirs, excel_df, preprocessor, transform=None, target_size=(224, 224)):
        """
        image_dirs: 包含三个类别的图像目录字典
        excel_df: 包含人口统计特征和标签的DataFrame
        preprocessor: 预处理管道（用于人口统计特征）
        transform: 图像变换
        target_size: 图像目标尺寸
        """
        self.image_paths = []
        self.labels = []
        self.target_size = target_size
        self.transform = transform
        self.preprocessor = preprocessor
        
        # 构建图像路径和标签列表
        for class_id, dir_path in image_dirs.items():
            if not os.path.exists(dir_path):
                continue
            for img_file in os.listdir(dir_path):
                if img_file.endswith(('.jpg', '.jpeg', '.png')):
                    # 提取eid_ckd从文件名
                    eid_str = img_file.split('_')[1] if img_file.startswith(('left_', 'right_')) else img_file.split('_')[0]
                    try:
                        eid = int(eid_str)
                    except ValueError:
                        continue
                    
                    # 查找对应的人口统计数据
                    if eid in excel_df['eid_ckd'].values:
                        self.image_paths.append(os.path.join(dir_path, img_file))
                        self.labels.append(class_id)
        
        # 创建eid到人口统计数据的映射
        self.eid_demo_map = {row['eid_ckd']: row for _, row in excel_df.iterrows()}
        
        # 提取人口统计特征列
        self.demo_cols = ['townsend', 'smokingc', 'drinkc', 'bmi', 'baselineage', 
                         'ethnic', 'edu', 'hbp', 'dmstatus', 'hyperlipidemia', 'gender']
    
    def __len__(self):
        return len(self.image_paths)
    
    def __getitem__(self, idx):
        # 加载和预处理图像
        img_path = self.image_paths[idx]
        try:
            with Image.open(img_path).convert('RGB') as img:
                # 调整大小并中心裁剪
                width, height = img.size
                ratio = min(self.target_size[0]/width, self.target_size[1]/height)
                new_size = (int(width * ratio), int(height * ratio))
                img = img.resize(new_size, Image.LANCZOS)
                
                left = (new_size[0] - self.target_size[0]) // 2
                top = (new_size[1] - self.target_size[1]) // 2
                img = img.crop((left, top, left + self.target_size[0], top + self.target_size[1]))
                
                # 应用变换
                if self.transform:
                    img = self.transform(img)
                
                # 标准化
                img = transforms.Normalize(mean=[0.485, 0.456, 0.406], 
                                         std=[0.229, 0.224, 0.225])(img)
        except Exception as e:
            print(f"处理图像 {img_path} 出错: {e}")
            return None
        
        # 提取和预处理人口统计特征
        img_file = os.path.basename(img_path)
        eid_str = img_file.split('_')[1] if img_file.startswith(('left_', 'right_')) else img_file.split('_')[0]
        try:
            eid = int(eid_str)
        except ValueError:
            return None
            
        if eid not in self.eid_demo_map:
            return None
            
        demo_data = self.eid_demo_map[eid]
        demo_df = pd.DataFrame([demo_data[self.demo_cols].values], columns=self.demo_cols)
        demo_features = self.preprocessor.transform(demo_df)[0]
        demo_features = torch.tensor(demo_features, dtype=torch.float32)
        
        # 获取标签
        label = torch.tensor(self.labels[idx], dtype=torch.long)
        
        return img, demo_features, label

# 4. 定义双模态RetFound模型
class DualModalRetFound(nn.Module):
    def __init__(self, demo_feature_dim, num_classes=3, freeze_retfound=True):
        super(DualModalRetFound, self).__init__()
        # 加载预训练的RetFound模型
        self.retfound = timm.create_model(
            'vit_large_patch16_224_retfound',
            pretrained=True,
            num_classes=0,  # 不加载分类头
            in_chans=3
        )
        self.image_feature_dim = self.retfound.num_features
        
        # 冻结RetFound的参数（微调初期）
        if freeze_retfound:
            for param in self.retfound.parameters():
                param.requires_grad = False
        
        # 人口统计特征处理分支
        self.demo_branch = nn.Sequential(
            nn.Linear(demo_feature_dim, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, self.image_feature_dim)
        )
        
        # 特征融合层
        self.fusion = nn.Sequential(
            nn.Linear(self.image_feature_dim * 2, self.image_feature_dim),
            nn.BatchNorm1d(self.image_feature_dim),
            nn.ReLU(),
            nn.Dropout(0.3)
        )
        
        # 分类头
        self.classifier = nn.Linear(self.image_feature_dim, num_classes)
        
    def forward(self, images, demo_features):
        # 提取图像特征
        img_feat = self.retfound(images)
        
        # 处理人口统计特征
        demo_feat = self.demo_branch(demo_features)
        
        # 融合特征
        fused_feat = torch.cat([img_feat, demo_feat], dim=1)
        fused_feat = self.fusion(fused_feat)
        
        # 分类
        logits = self.classifier(fused_feat)
        return logits

# 5. 训练和评估函数
def train_model(model, train_loader, val_loader, criterion, optimizer, num_epochs=20, device='cuda'):
    """训练模型并在验证集上评估"""
    model.to(device)
    best_val_acc = 0.0
    best_model_weights = None
    
    # 记录训练过程
    history = {
        'train_loss': [], 'train_acc': [],
        'val_loss': [], 'val_acc': []
    }
    
    for epoch in range(num_epochs):
        print(f'Epoch {epoch+1}/{num_epochs}')
        print('-' * 10)
        
        # 训练阶段
        model.train()
        running_loss = 0.0
        running_corrects = 0
        
        for batch in train_loader:
            if batch is None:
                continue
                
            images, demo_feats, labels = batch
            images = images.to(device)
            demo_feats = demo_feats.to(device)
            labels = labels.to(device)
            
            # 清零梯度
            optimizer.zero_grad()
            
            # 前向传播
            with torch.set_grad_enabled(True):
                outputs = model(images, demo_feats)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                # 反向传播和优化
                loss.backward()
                optimizer.step()
            
            # 统计
            running_loss += loss.item() * images.size(0)
            running_corrects += torch.sum(preds == labels.data)
        
        epoch_loss = running_loss / len(train_loader.dataset)
        epoch_acc = running_corrects.double() / len(train_loader.dataset)
        
        history['train_loss'].append(epoch_loss)
        history['train_acc'].append(epoch_acc.item())
        
        print(f'Train Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')
        
        # 验证阶段
        model.eval()
        running_val_loss = 0.0
        running_val_corrects = 0
        all_preds = []
        all_labels = []
        
        for batch in val_loader:
            if batch is None:
                continue
                
            images, demo_feats, labels = batch
            images = images.to(device)
            demo_feats = demo_feats.to(device)
            labels = labels.to(device)
            
            # 前向传播
            with torch.set_grad_enabled(False):
                outputs = model(images, demo_feats)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
            
            # 统计
            running_val_loss += loss.item() * images.size(0)
            running_val_corrects += torch.sum(preds == labels.data)
            
            # 收集所有预测和标签用于详细评估
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
        
        val_loss = running_val_loss / len(val_loader.dataset)
        val_acc = running_val_corrects.double() / len(val_loader.dataset)
        
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc.item())
        
        print(f'Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}')
        
        # 保存最佳模型
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_model_weights = model.state_dict()
        
        # 打印详细分类报告
        print("\n验证集分类报告:")
        print(classification_report(all_labels, all_preds, target_names=['无抑郁', '基线抑郁', '新发抑郁']))
        
        # 绘制混淆矩阵
        cm = confusion_matrix(all_labels, all_preds)
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                   xticklabels=['无抑郁', '基线抑郁', '新发抑郁'],
                   yticklabels=['无抑郁', '基线抑郁', '新发抑郁'])
        plt.xlabel('预测标签')
        plt.ylabel('真实标签')
        plt.title(f'第{epoch+1}轮混淆矩阵')
        plt.show()
    
    # 加载最佳模型权重
    model.load_state_dict(best_model_weights)
    return model, history

# 6. 主函数
def main():
    # 配置路径
    EXCEL_PATH = "D:/LingYi/0820_ukb_depression_fundus.xlsx"
    IMAGE_BASE_DIR = "D:/LingYi/data_pre_depression_by_right_eye"
    SAVE_DIR = "D:/LingYi/retfound_results"
    os.makedirs(SAVE_DIR, exist_ok=True)
    
    # 读取Excel数据
    df = pd.read_excel(EXCEL_PATH)
    print(f"成功读取Excel数据，共 {len(df)} 条记录")
    
    # 按eid_ckd拆分训练集、验证集和测试集
    eids = df['eid_ckd'].unique()
    train_eids, temp_eids = train_test_split(eids, test_size=0.3, random_state=SEED)
    val_eids, test_eids = train_test_split(temp_eids, test_size=0.5, random_state=SEED)
    
    # 拆分数据集
    train_df = df[df['eid_ckd'].isin(train_eids)]
    val_df = df[df['eid_ckd'].isin(val_eids)]
    test_df = df[df['eid_ckd'].isin(test_eids)]
    
    # 填充缺失值
    train_df_filled, fill_params = fill_missing_values(train_df, is_train=True)
    val_df_filled = fill_missing_values(val_df, is_train=False, fill_params=fill_params)
    test_df_filled = fill_missing_values(test_df, is_train=False, fill_params=fill_params)
    
    # 保存填充参数
    # 将NumPy类型转换为Python原生类型
    def convert_numpy_types(obj):
        if isinstance(obj, np.integer):
            return int(obj)
        elif isinstance(obj, np.floating):
            return float(obj)
        elif isinstance(obj, np.ndarray):
            return obj.tolist()
        else:
            return obj

    # 转换所有参数类型
    fill_params_converted = {k: convert_numpy_types(v) for k, v in fill_params.items()}

    # 保存转换后的参数
    with open(os.path.join(SAVE_DIR, 'fill_params.json'), 'w') as f:
        json.dump(fill_params_converted, f, indent=4)
    
    # 为每个样本生成三分类标签
    train_df_filled['label'] = train_df_filled.apply(get_depression_class, axis=1)
    val_df_filled['label'] = val_df_filled.apply(get_depression_class, axis=1)
    test_df_filled['label'] = test_df_filled.apply(get_depression_class, axis=1)
    
    # 定义人口统计特征预处理管道
    numeric_features = ['townsend', 'bmi', 'baselineage']
    categorical_features = ['gender', 'smokingc', 'drinkc', 'ethnic', 'edu', 
                           'hbp', 'dmstatus', 'hyperlipidemia']
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(sparse_output=False, drop='first'), categorical_features)
        ])
    
    # 拟合预处理管道（仅使用训练集）
    preprocessor.fit(train_df_filled[numeric_features + categorical_features])
    demo_feature_dim = preprocessor.transform(train_df_filled[numeric_features + categorical_features]).shape[1]
    print(f"人口统计特征预处理后维度: {demo_feature_dim}")
    
    # 定义图像变换
    train_transform = transforms.Compose([
        transforms.RandomHorizontalFlip(p=0.5),
        transforms.RandomRotation(degrees=(-5, 5)),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.1),
        transforms.ToTensor()
    ])
    
    val_test_transform = transforms.Compose([
        transforms.ToTensor()
    ])
    
    # 定义训练/验证/测试集的图像目录
    def get_image_dirs(base_dir, dataset_type):
        return {
            0: os.path.join(base_dir, dataset_type, 'class_0_non_depression'),
            1: os.path.join(base_dir, dataset_type, 'class_1_baseline_dep'),
            2: os.path.join(base_dir, dataset_type, 'class_2_incident_dep')
        }
    
    train_image_dirs = get_image_dirs(IMAGE_BASE_DIR, 'train')
    val_image_dirs = get_image_dirs(IMAGE_BASE_DIR, 'val')
    test_image_dirs = get_image_dirs(IMAGE_BASE_DIR, 'test')
    
    # 创建数据集
    train_dataset = DualModalDataset(
        train_image_dirs, train_df_filled, preprocessor, 
        transform=train_transform, target_size=(224, 224)
    )
    val_dataset = DualModalDataset(
        val_image_dirs, val_df_filled, preprocessor, 
        transform=val_test_transform, target_size=(224, 224)
    )
    test_dataset = DualModalDataset(
        test_image_dirs, test_df_filled, preprocessor, 
        transform=val_test_transform, target_size=(224, 224)
    )
    
    print(f"训练集样本数: {len(train_dataset)}")
    print(f"验证集样本数: {len(val_dataset)}")
    print(f"测试集样本数: {len(test_dataset)}")
    
    # 创建数据加载器
    batch_size = 16
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=4)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=4)
    
    # 初始化模型
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"使用设备: {device}")
    
    model = DualModalRetFound(
        demo_feature_dim=demo_feature_dim,
        num_classes=3,
        freeze_retfound=True  # 初始冻结RetFound
    )
    
    # 定义损失函数和优化器
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=1e-4, weight_decay=1e-5)
    
    # 训练模型
    print("\n开始训练模型...")
    model, history = train_model(
        model, train_loader, val_loader, 
        criterion, optimizer, 
        num_epochs=20,
        device=device
    )
    
    # 保存模型
    torch.save(model.state_dict(), os.path.join(SAVE_DIR, 'dual_modal_retfound.pth'))
    
    # 绘制训练历史
    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(history['train_loss'], label='训练损失')
    plt.plot(history['val_loss'], label='验证损失')
    plt.title('损失曲线')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    
    plt.subplot(1, 2, 2)
    plt.plot(history['train_acc'], label='训练准确率')
    plt.plot(history['val_acc'], label='验证准确率')
    plt.title('准确率曲线')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(SAVE_DIR, 'training_history.png'))
    plt.show()
    
    # 在测试集上评估
    print("\n在测试集上评估模型...")
    model.eval()
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        for batch in test_loader:
            if batch is None:
                continue
                
            images, demo_feats, labels = batch
            images = images.to(device)
            demo_feats = demo_feats.to(device)
            
            outputs = model(images, demo_feats)
            _, preds = torch.max(outputs, 1)
            
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.numpy())
    
    # 打印测试集结果
    print("\n测试集分类报告:")
    print(classification_report(all_labels, all_preds, target_names=['无抑郁', '基线抑郁', '新发抑郁']))
    
    # 绘制测试集混淆矩阵
    cm = confusion_matrix(all_labels, all_preds)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
               xticklabels=['无抑郁', '基线抑郁', '新发抑郁'],
               yticklabels=['无抑郁', '基线抑郁', '新发抑郁'])
    plt.xlabel('预测标签')
    plt.ylabel('真实标签')
    plt.title('测试集混淆矩阵')
    plt.savefig(os.path.join(SAVE_DIR, 'test_confusion_matrix.png'))
    plt.show()
    
    test_acc = accuracy_score(all_labels, all_preds)
    print(f"测试集准确率: {test_acc:.4f}")

if __name__ == "__main__":
    main()


成功读取Excel数据，共 43445 条记录
人口统计特征预处理后维度: 13
训练集样本数: 0
验证集样本数: 0
测试集样本数: 0


ValueError: num_samples should be a positive integer value, but got num_samples=0

In [9]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
# import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import (classification_report, confusion_matrix, accuracy_score,
                             f1_score, recall_score, precision_score)
from sklearn.model_selection import GridSearchCV, cross_val_score
import joblib

import lightgbm as lgb
from imblearn.pipeline import Pipeline as ImbPipeline
from imblearn.combine import SMOTETomek  # 结合过采样和欠采样

# 设置中文显示
plt.rcParams["font.family"] = ["SimHei", "WenQuanYi Micro Hei", "Heiti TC"]
plt.rcParams["axes.unicode_minus"] = False  # 正确显示负号

def load_and_preprocess_data(excel_path):
    """加载数据并进行预处理"""
    # 读取Excel数据
    df = pd.read_excel(excel_path)
    print(f"原始数据形状: {df.shape}")
    
    # 定义人口统计学特征列（根据实际数据调整）
    demographic_cols = [
        'townsend', 'smokingc', 'drinkc', 'bmi', 
        'baselineage', 'ethnic', 'edu', 'hbp', 'dmstatus', 'hyperlipidemia', 'gender'
    ]
    
    # 检查并保留存在的列
    existing_cols = [col for col in demographic_cols if col in df.columns]
    print(f"使用的人口统计学特征: {existing_cols}")
    
    # 三分类标签函数
    def classify_depression(row):
        """
        三分类判断：
        - 0: 无抑郁
        - 1: 基线抑郁
        - 2: 新发抑郁
        """
#         baseline_depressed = (pd.notna(row['new_totdepress']) and row['new_totdepress'] == 1) or \
        baseline_depressed =                    (pd.notna(row['baseline_depression']) and row['baseline_depression'] == '是')
        
        incident_depressed = pd.notna(row['incident_depression']) and row['incident_depression'] == '是'
        
        if baseline_depressed:
            return 1  # 基线抑郁
        elif not baseline_depressed and incident_depressed:
            return 2  # 新发抑郁
        else:
            return 0  # 无抑郁
    
    # 创建标签列
    df['label'] = df.apply(classify_depression, axis=1)
    
    # 检查标签分布
    print("\n标签分布:")
    print(df['label'].value_counts().sort_index())
    print(f"0: 无抑郁, 1: 基线抑郁, 2: 新发抑郁")
    
    # 分离特征和标签
    X = df[existing_cols].copy()
    y = df['label'].copy()
    
    # 填充缺失值（修正警告版本）
    numeric_cols = X.select_dtypes(include=['number']).columns.tolist()
    categorical_cols = X.select_dtypes(exclude=['number']).columns.tolist()

    # 数值型用中位数填充（不使用inplace=True）
    for col in numeric_cols:
        X[col] = X[col].fillna(X[col].median())  # 直接赋值替换

    # 类别型用众数填充（不使用inplace=True）
    for col in categorical_cols:
        X[col] = X[col].fillna(X[col].mode()[0])  # 直接赋值替换
    
    return X, y, numeric_cols, categorical_cols

def split_dataset(X, y, test_size=0.2, val_size=0.15, random_state=42):
    """分割训练集、验证集和测试集"""
    # 先分割训练集和临时集
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=test_size + val_size, random_state=random_state, stratify=y
    )
    
    # 从临时集中分割验证集和测试集
    val_ratio = val_size / (test_size + val_size)
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=1 - val_ratio, random_state=random_state, stratify=y_temp
    )
    
    print(f"\n数据集分割:")
    print(f"训练集: {X_train.shape}, 验证集: {X_val.shape}, 测试集: {X_test.shape}")
    return X_train, X_val, X_test, y_train, y_val, y_test

def build_preprocessing_pipeline(numeric_cols, categorical_cols):
    """构建预处理管道"""
    # 定义预处理步骤
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_cols),  # 数值特征标准化
            ('cat', OneHotEncoder(drop='first', handle_unknown='ignore'), categorical_cols)  # 类别特征独热编码
        ])
    
    return preprocessor

def train_random_forest(X_train, y_train, X_val, y_val, preprocessor):
    """训练随机森林模型并进行超参数调优"""
    # 构建包含预处理和LightGBM模型的管道
    pipeline = ImbPipeline([
        ('preprocessor', preprocessor),
        ('sampler', SMOTETomek(random_state=42)),  # 对训练集进行重采样
        ('classifier', lgb.LGBMClassifier(
            # 自定义类别权重：无抑郁=1，基线抑郁=15，新发抑郁=25（根据样本比例设置）
            class_weight={0:1, 1:15, 2:25},
            objective='multiclass',
            num_class=3,
            random_state=42,
            verbose=-1
        ))
        
    ])

    # 修正后的LightGBM超参数搜索空间
    param_grid = {
        'classifier__n_estimators': [400],  # 增加树数量，避免欠拟合
        'classifier__learning_rate': [0.05],
        'classifier__max_depth': [8],
        'classifier__num_leaves': [63],  # 控制树复杂度
        'classifier__min_child_samples': [5],  # 防止过拟合
        'classifier__subsample': [0.8],
        'classifier__colsample_bytree': [1.0]  # 特征采样，增加多样性
    }
    
    # 网格搜索
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=5, scoring='f1_macro', n_jobs=-1, verbose=1
    )
    
    print("\n开始模型训练和超参数搜索...")
    grid_search.fit(X_train, y_train)
    
    print(f"最佳参数: {grid_search.best_params_}")
    print(f"训练集最佳F1分数: {grid_search.best_score_:.4f}")
    
    # 在验证集上评估
    val_preds = grid_search.predict(X_val)
    print("\n验证集评估:")
    print(classification_report(y_val, val_preds, target_names=['无抑郁', '基线抑郁', '新发抑郁']))
    
    return grid_search.best_estimator_

def evaluate_model(model, X_test, y_test, feature_names):
    """评估模型并可视化结果"""
    # 在测试集上预测
    y_pred = model.predict(X_test)
    
    # 计算评估指标
    print("\n测试集评估:")
    print(f"准确率: {accuracy_score(y_test, y_pred):.4f}")
    print(f"宏平均F1分数: {f1_score(y_test, y_pred, average='macro'):.4f}")
    print(f"宏平均精确率: {precision_score(y_test, y_pred, average='macro'):.4f}")
    print(f"宏平均召回率: {recall_score(y_test, y_pred, average='macro'):.4f}")
    
    # 详细分类报告
    print("\n分类报告:")
    print(classification_report(
        y_test, y_pred, 
        target_names=['无抑郁', '基线抑郁', '新发抑郁']
    ))
    
    # 混淆矩阵
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(
        cm, annot=True, fmt='d', cmap='Blues', 
        xticklabels=['无抑郁', '基线抑郁', '新发抑郁'],
        yticklabels=['无抑郁', '基线抑郁', '新发抑郁']
    )
    plt.xlabel('预测标签')
    plt.ylabel('真实标签')
    plt.title('测试集混淆矩阵')
    plt.tight_layout()
    plt.savefig('confusion_matrix.png')
    plt.show()
    
    # 特征重要性（如果是树模型）
    if hasattr(model['classifier'], 'feature_importances_'):
        # 获取预处理后的特征名
        ohe = model.named_steps['preprocessor'].named_transformers_['cat']
        cat_features = list(ohe.get_feature_names_out(categorical_cols))
        all_features = numeric_cols + cat_features
        
        # 提取特征重要性
        importances = model['classifier'].feature_importances_
        indices = np.argsort(importances)[::-1]
        
        # 可视化特征重要性
        plt.figure(figsize=(12, 8))
        plt.barh(range(len(indices[:10])), importances[indices[:10]], color='b', align='center')
        plt.yticks(range(len(indices[:10])), [all_features[i] for i in indices[:10]])
        plt.xlabel('特征重要性')
        plt.title('随机森林特征重要性（前10名）')
        plt.tight_layout()
        plt.savefig('feature_importance.png')
        plt.show()

def main():
    # 配置路径
    EXCEL_PATH = "D:/LingYi/0820_ukb_depression_fundus.xlsx"  # 替换为你的Excel路径
    MODEL_SAVE_PATH = "demographic_rf_model.pkl"
    
    # 加载和预处理数据
    X, y, numeric_cols, categorical_cols = load_and_preprocess_data(EXCEL_PATH)
    
    # 分割数据集
    X_train, X_val, X_test, y_train, y_val, y_test = split_dataset(X, y)
    
    # 构建预处理管道
    preprocessor = build_preprocessing_pipeline(numeric_cols, categorical_cols)
    
    # 训练模型
    best_model = train_random_forest(X_train, y_train, X_val, y_val, preprocessor)
    
    # 评估模型
    evaluate_model(best_model, X_test, y_test, X.columns)
    
    # 保存模型
    joblib.dump(best_model, MODEL_SAVE_PATH)
    print(f"\n模型已保存至: {MODEL_SAVE_PATH}")
    
    # 交叉验证
    cv_scores = cross_val_score(best_model, X, y, cv=5, scoring='f1_macro')
    print(f"\n5折交叉验证F1分数: {cv_scores.mean():.4f} ± {cv_scores.std():.4f}")

if __name__ == "__main__":
    main()


原始数据形状: (43445, 18)
使用的人口统计学特征: ['townsend', 'smokingc', 'drinkc', 'bmi', 'baselineage', 'ethnic', 'edu', 'hbp', 'dmstatus', 'hyperlipidemia', 'gender']

标签分布:
label
0    39503
1     2540
2     1402
Name: count, dtype: int64
0: 无抑郁, 1: 基线抑郁, 2: 新发抑郁

数据集分割:
训练集: (28239, 11), 验证集: (6516, 11), 测试集: (8690, 11)

开始模型训练和超参数搜索...
Fitting 5 folds for each of 1 candidates, totalling 5 fits


D:\Anaconda\envs\pytorch_env\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
D:\Anaconda\envs\pytorch_env\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(
D:\Anaconda\envs\pytorch_env\lib\site-packages\sklearn\base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


最佳参数: {'classifier__colsample_bytree': 1.0, 'classifier__learning_rate': 0.05, 'classifier__max_depth': 8, 'classifier__min_child_samples': 5, 'classifier__n_estimators': 400, 'classifier__num_leaves': 63, 'classifier__subsample': 0.8}
训练集最佳F1分数: 0.2832


D:\Anaconda\envs\pytorch_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



验证集评估:
              precision    recall  f1-score   support

         无抑郁       0.92      0.46      0.61      5925
        基线抑郁       0.08      0.36      0.12       381
        新发抑郁       0.03      0.27      0.06       210

    accuracy                           0.45      6516
   macro avg       0.34      0.36      0.27      6516
weighted avg       0.84      0.45      0.57      6516



D:\Anaconda\envs\pytorch_env\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but LGBMClassifier was fitted with feature names
  warnings.warn(



测试集评估:
准确率: 0.4574
宏平均F1分数: 0.2756
宏平均精确率: 0.3490
宏平均召回率: 0.4000

分类报告:
              precision    recall  f1-score   support

         无抑郁       0.93      0.47      0.62      7901
        基线抑郁       0.08      0.35      0.12       508
        新发抑郁       0.05      0.38      0.08       281

    accuracy                           0.46      8690
   macro avg       0.35      0.40      0.28      8690
weighted avg       0.85      0.46      0.57      8690



NameError: name 'sns' is not defined

<Figure size 1000x800 with 0 Axes>

In [2]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import glob
from transformers import AutoImageProcessor  # RETFound官方推荐的图像预处理工具

class BilateralRetinaDataset(Dataset):
    def __init__(self, root_dir, split="train", num_classes=2, image_processor=None):
        """
        适配你生成的双眼数据集目录结构：
        root_dir/
        ├─ train/
        │  ├─ bilateral/
        │  │  ├─ class_depression/
        │  │  │  ├─ bilateral_1001_0_left.jpg
        │  │  │  ├─ bilateral_1001_0_right.jpg
        │  │  └─ class_non_depression/
        ├─ val/...
        """
        self.root = os.path.join(root_dir, split, "bilateral")  # 双眼配对目录
        self.num_classes = num_classes
        self.image_processor = image_processor  # RETFound的图像预处理（替代自定义transform）
        self.class_map = {"class_depression": 1, "class_non_depression": 0}  # 标签映射
        self.pair_list = self._collect_bilateral_pairs()  # 收集所有双眼图像对

    def _collect_bilateral_pairs(self):
        """按文件名前缀匹配左眼和右眼图像对（核心：对齐你生成的文件名格式）"""
        pair_list = []
        # 遍历抑郁/非抑郁两类目录
        for cls_name, cls_label in self.class_map.items():
            cls_dir = os.path.join(self.root, cls_name)
            if not os.path.exists(cls_dir):
                continue
            # 找到所有左眼图像，通过文件名替换匹配右眼图像
            left_imgs = glob.glob(os.path.join(cls_dir, "*_left.jpg"))
            for left_path in left_imgs:
                # 替换_left.jpg为_right.jpg，匹配同组右眼图像
                right_path = left_path.replace("_left.jpg", "_right.jpg")
                if os.path.exists(right_path):
                    pair_list.append((left_path, right_path, cls_label))
        print(f"[{self.root}] 收集到 {len(pair_list)} 个双眼图像对")
        return pair_list

    def __len__(self):
        return len(self.pair_list)

    def __getitem__(self, idx):
        left_path, right_path, label = self.pair_list[idx]
        # 加载图像（RGB格式，与预处理的224×224一致）
        left_img = Image.open(left_path).convert("RGB")
        right_img = Image.open(right_path).convert("RGB")
        
        # 使用RETFound官方ImageProcessor预处理（避免与预训练权重冲突）
        # 输出为模型所需的pixel_values（tensor格式，shape: (3, 224, 224)）
        left_processed = self.image_processor(
            left_img, return_tensors="pt", do_resize=False, do_center_crop=False
        )["pixel_values"].squeeze(0)  # 你已提前裁剪为224×224，关闭resize/crop
        right_processed = self.image_processor(
            right_img, return_tensors="pt", do_resize=False, do_center_crop=False
        )["pixel_values"].squeeze(0)
        
        # 返回字典格式，与模型输入（batch["left_img"]等）完全匹配
        return {
            "left_img": left_processed,
            "right_img": right_processed,
            "label": torch.tensor(label, dtype=torch.long)
        }

# 初始化DataLoader（训练/验证/测试集通用）
def build_bilateral_dataloader(args, split="train"):
    # 加载RETFound官方图像处理器（确保与预训练权重的预处理一致）
    from transformers import AutoImageProcessor
    image_processor = AutoImageProcessor.from_pretrained(args.finetune)
    
    # 初始化数据集
    dataset = BilateralRetinaDataset(
        root_dir=args.data_path,
        split=split,
        num_classes=args.nb_classes,
        image_processor=image_processor
    )
    
    # 构建DataLoader
    return DataLoader(
        dataset,
        batch_size=args.batch_size,
        shuffle=(split == "train"),  # 训练集打乱，验证/测试集不打乱
        num_workers=args.num_workers,
        pin_memory=True
    )

D:\Anaconda\envs\retfound\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
import torch
import torch.nn as nn
from transformers import AutoConfig, AutoModel

def build_model(args):
    model_type = args.model
    pretrained_path = args.finetune  # RETFound预训练权重路径（如"microsoft/retfound-mae-base"）
    
    if model_type == "RETFound_mae_bilateral":
        # 1. 加载RETFound MAE预训练配置和骨干网络
        # 关键：指定output_hidden_states=True，确保能获取last_hidden_state
        config = AutoConfig.from_pretrained(pretrained_path, output_hidden_states=True)
        base_model = AutoModel.from_pretrained(pretrained_path, config=config)
        feature_dim = config.hidden_size  # RETFound的CLS特征维度（如768/1024）
        
        # 2. 定义双眼特征融合模型（沿用你的核心逻辑，补充细节）
        class BilateralRETFoundModel(nn.Module):
            def __init__(self, base_model, feature_dim, num_classes, drop_path=0.1):
                super().__init__()
                self.base_model = base_model
                self.num_classes = num_classes
                
                # 双眼特征融合模块（参考论文：自适应线性融合+BN+Dropout）
                self.fusion = nn.Sequential(
                    nn.Linear(feature_dim * 2, feature_dim),  # 2*feat_dim → feat_dim
                    nn.BatchNorm1d(feature_dim),  # 批量归一化，稳定训练
                    nn.ReLU(inplace=True),  # 激活函数
                    nn.Dropout(drop_path)  # 防止过拟合，与args.drop_path对齐
                )
                
                # 分类头（与RETFound微调任务一致，使用线性层）
                self.class_head = nn.Linear(feature_dim, num_classes)
                
                # 3. 冻结预训练骨干网络底层（可选，小样本场景推荐）
                self._freeze_backbone_layers(freeze_ratio=0.7)  # 冻结70%的Transformer层

            def _freeze_backbone_layers(self, freeze_ratio=0.7):
                """冻结RETFound骨干网络的底层Transformer层（避免权重破坏）"""
                num_layers = len(self.base_model.encoder.layer)  # Transformer层总数
                freeze_layers = int(num_layers * freeze_ratio)
                for i, layer in enumerate(self.base_model.encoder.layer):
                    if i < freeze_layers:
                        for param in layer.parameters():
                            param.requires_grad = False
                print(f"冻结RETFound底层 {freeze_layers}/{num_layers} 个Transformer层")

            def extract_single_feat(self, img_tensor):
                """提取单眼图像的CLS特征（适配RETFound输出格式）"""
                # img_tensor: (batch_size, 3, 224, 224)
                outputs = self.base_model(pixel_values=img_tensor)
                # 获取CLS Token的特征（last_hidden_state的第0个token）
                cls_feat = outputs.last_hidden_state[:, 0, :]  # (batch_size, feature_dim)
                return cls_feat

            def forward(self, batch):
                """
                输入：batch字典（left_img/right_img/label）
                输出：logits（预测概率）、labels（真实标签）
                """
                # 从batch中获取双眼图像和标签
                left_imgs = batch["left_img"]  # (batch_size, 3, 224, 224)
                right_imgs = batch["right_img"]  # (batch_size, 3, 224, 224)
                labels = batch["label"]  # (batch_size,)
                
                # 独立提取双眼特征
                left_feat = self.extract_single_feat(left_imgs)  # (bs, feat_dim)
                right_feat = self.extract_single_feat(right_imgs)  # (bs, feat_dim)
                
                # 特征融合：拼接→线性压缩→分类
                fused_feat = torch.cat([left_feat, right_feat], dim=1)  # (bs, 2*feat_dim)
                fused_feat = self.fusion(fused_feat)  # (bs, feat_dim)
                logits = self.class_head(fused_feat)  # (bs, num_classes)
                
                return logits, labels
        
        # 4. 初始化双眼模型
        model = BilateralRETFoundModel(
            base_model=base_model,
            feature_dim=feature_dim,
            num_classes=args.nb_classes,
            drop_path=args.drop_path  # 从args传入Dropout比例，确保配置统一
        )
        print(f"成功初始化双眼RETFound模型，分类头维度：{feature_dim}→{args.nb_classes}")
    
    else:
        # 保留原单眼模型逻辑（兼容其他模型类型）
        model = create_retfound_mae_model(args)
    
    return model

In [4]:
def train_one_epoch(args, model, dataloader, criterion, optimizer, device):
    model.train()
    total_loss = 0.0
    total_correct = 0
    total_samples = 0
    
    for batch_idx, batch in enumerate(dataloader):
        # 1. 将batch数据移到GPU（若使用GPU）
        batch = {k: v.to(device) for k, v in batch.items()}
        
        # 2. 模型前向传播（使用你定义的forward逻辑）
        logits, labels = model(batch)
        
        # 3. 计算损失（二分类任务用CrossEntropyLoss）
        loss = criterion(logits, labels)
        
        # 4. 反向传播与优化
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # 5. 统计训练指标
        total_loss += loss.item() * labels.size(0)
        preds = torch.argmax(logits, dim=1)
        total_correct += (preds == labels).sum().item()
        total_samples += labels.size(0)
        
        # 打印训练日志（每10个batch打印一次）
        if (batch_idx + 1) % args.log_interval == 0:
            avg_loss = total_loss / total_samples
            avg_acc = total_correct / total_samples
            print(f"Train Batch [{batch_idx+1}/{len(dataloader)}] | Loss: {avg_loss:.4f} | Acc: {avg_acc:.4f}")
    
    # 返回epoch级指标
    epoch_avg_loss = total_loss / total_samples
    epoch_avg_acc = total_correct / total_samples
    return epoch_avg_loss, epoch_avg_acc

# 主训练函数（简化版）
def main(args):
    # 初始化设备（CPU/GPU）
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    
    # 1. 构建双眼DataLoader
    train_loader = build_bilateral_dataloader(args, split="train")
    val_loader = build_bilateral_dataloader(args, split="val")
    
    # 2. 构建模型（使用你的双眼模型）
    model = build_model(args).to(device)
    
    # 3. 初始化损失函数和优化器
    criterion = nn.CrossEntropyLoss()  # 二分类任务
    optimizer = torch.optim.AdamW(
        model.parameters(),
        lr=args.lr,
        weight_decay=args.weight_decay  # 正则化，防止过拟合
    )
    
    # 4. 开始训练
    for epoch in range(args.epochs):
        print(f"\n=== Epoch [{epoch+1}/{args.epochs}] ===")
        # 训练一个epoch
        train_loss, train_acc = train_one_epoch(args, model, train_loader, criterion, optimizer, device)
        # 验证一个epoch（逻辑类似train_one_epoch，仅需关闭梯度计算）
        val_loss, val_acc = validate_one_epoch(args, model, val_loader, criterion, device)
        
        # 保存最佳模型（按验证集准确率）
        if val_acc > args.best_val_acc:
            args.best_val_acc = val_acc
            torch.save(model.state_dict(), os.path.join(args.save_dir, "best_bilateral_model.pth"))
            print(f"保存最佳模型，验证准确率：{val_acc:.4f}")

# 命令行参数解析（补充必要参数）
def parse_args():
    import argparse
    parser = argparse.ArgumentParser()
    # 模型配置
    parser.add_argument("--model", type=str, default="RETFound_mae_bilateral", help="模型类型")
    parser.add_argument("--finetune", type=str, required=True, help="RETFound预训练权重路径")
    parser.add_argument("--nb_classes", type=int, default=2, help="分类数（抑郁/非抑郁）")
    parser.add_argument("--drop_path", type=float, default=0.1, help="Dropout比例")
    # 数据配置
    parser.add_argument("--data_path", type=str, required=True, help="双眼数据集根目录")
    parser.add_argument("--batch_size", type=int, default=16, help="批次大小")
    parser.add_argument("--num_workers", type=int, default=4, help="数据加载线程数")
    # 训练配置
    parser.add_argument("--epochs", type=int, default=30, help="训练轮数")
    parser.add_argument("--lr", type=float, default=1e-4, help="学习率")
    parser.add_argument("--weight_decay", type=float, default=1e-5, help="权重衰减")
    parser.add_argument("--log_interval", type=int, default=10, help="日志打印间隔")
    parser.add_argument("--save_dir", type=str, default="./save_models", help="模型保存目录")
    return parser.parse_args()

if __name__ == "__main__":
    args = parse_args()
    # 创建模型保存目录
    os.makedirs(args.save_dir, exist_ok=True)
    # 初始化最佳验证准确率
    args.best_val_acc = 0.0
    # 启动训练
    main(args)

usage: ipykernel_launcher.py [-h] [--model MODEL] --finetune FINETUNE [--nb_classes NB_CLASSES]
                             [--drop_path DROP_PATH] --data_path DATA_PATH [--batch_size BATCH_SIZE]
                             [--num_workers NUM_WORKERS] [--epochs EPOCHS] [--lr LR] [--weight_decay WEIGHT_DECAY]
                             [--log_interval LOG_INTERVAL] [--save_dir SAVE_DIR]
ipykernel_launcher.py: error: the following arguments are required: --finetune, --data_path


SystemExit: 2

D:\Anaconda\envs\retfound\Lib\site-packages\IPython\core\interactiveshell.py:3707: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
